In [ ]:
!pip install checklist


In [ ]:
#imports

%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import sys
import numpy as np
import pandas as pd


import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

#model
model_name = "Constien/xmlRoberta_all_lang"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# sentiment analysis is a general name in Huggingface to load the pipeline for text classification tasks.
# set device=-1 if you don't have a gpu
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, framework="pt", device=0)

In [ ]:
editor = checklist.editor.Editor()
editor.tg

In [ ]:
suite = TestSuite()

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# test dataset
engset=pd.read_csv('/content/drive/MyDrive/IM/VTPAN_csv/VTPAN-test.csv')


In [ ]:
#predatory examples
pred_set=engset.loc[engset['label'] == 'predator']

In [ ]:
#non predatory examples
nopred_set=engset.loc[engset['label'] == 'non-predator']

In [ ]:
pred_set['segment_tur'] = pred_set['segment'].str[:1500]

In [ ]:
#pd.set_option('display.max_colwidth', None)
pred_set['segment_tur'].str.len().max()

In [ ]:
nopred_set['segment_tur'] = nopred_set['segment'].str[:1500]

In [ ]:
sentences = pred_set['segment_tur']

pdata = list(nlp.pipe(sentences))

In [ ]:
sentences_nopred = nopred_set['segment_tur']

pdata_nopred = list(nlp.pipe(sentences_nopred))

In [ ]:
sentences=list(sentences)
#sentences_nopred=list(sentences_nopred)

**Capability: neumeric reasoning**

In [ ]:
import re
def change_age(x, *args, **kwargs):
    # Returns empty or a list of strings with age changed
    age =  ['25', '30', '35', '40', '45', '50']
    age_replace= ['12', '13', '14','15', '16', '17']

    ret = []
    for p in age:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in age_replace if p != p2])
    return ret


In [ ]:
ret = Perturb.perturb(sentences_nopred, change_age)
#ret.sentences

In [ ]:
test = MFT(ret.data, labels=1)
suite.add(test,  'Age', 'Neumeric Reasoning', overwrite=True )

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
suite.summary()

In [ ]:
suite.run(pred_and_conf, overwrite=True)

In [ ]:
suite.summary()  # label 1, complete data 1 and 0

**Capability: vocabulary**

In [ ]:
#parent nouns
#parent_noun = ['dad', 'mom', 'aunt', 'grandma', 'uncle', 'grandfather', 'my grandmother', 'grandpa', 'my older sister', 'brother']
#editor.add_lexicon('parent_noun', parent_noun, overwrite=True)

names = ['my sweety', 'baby', 'babyy', 'babbby', 'little girl', 'sexy', 'darling','cutey', 'pussycat', 'angel', 'kiddo', 'kid', 'my girl', 'gorgeous']
editor.add_lexicon('names', names, overwrite=True)

In [ ]:
# positive adjectives
pos_adj = ['sexy','mesexy', 'great', 'allsome', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'attractive', 'girly', 'awesome', 'perfect', 'fun', 'cool', 'adorable', 'smart', 'exciting', 'sweet', 'wonderful']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)


In [ ]:
#negative_adj = ['lame', 'shitty', 'not worthy', 'unattractive', 'not sexy', 'not beautiful', 'not nice']
#editor.add_lexicon('neutral_adj', pos_adj, overwrite=True)


In [ ]:
# get model suggestion for possible neutral verbs
#print(', '.join(editor.suggest('I cant {a:mask} you today, {parent_noun} is home tonight.')[:40]))

to, with, for, tell, see, call, of, told, update, from, t, thank, write, tel, at, visit, -, show, called, hug, on, post, miss, w, help, hear, di, warn, near, without, leave, hold, msg, get, th, message, around, make, meet, bother


In [ ]:
#pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire', 'value']
#neutral_verb_present = ['call', 'see', 'visit', 'write', 'be near', 'come over', 'invite', 'msg', 'message', 'text', 'go online', 'log in', 'hug', 'touch', 'kiss', 'lick', 'smell']
#pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
#neutral_verb_past = ['called', 'saw', 'visited', 'wrote', 'was near', 'came over', 'invited', 'messaged', 'texted', 'came online', 'logged in', 'hugged', 'touched', 'kissed', 'licked', 'smelled']

#editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
#editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
#editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
#editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
#editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
#editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

Individual words

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
def add_NiWords(x):
    phrases = ['my sweety', 'baby', 'babyy', 'babbby', 'little girl', 'sexy', 'darling','cutie', 'angel', 'kiddo', 'kid', 'my girl', 'gorgeous']
    return ['%s %s' % (x, p) for p in phrases]

In [ ]:
monotonic_increasing = Expect.monotonic(label=1, increasing=False, tolerance=0.1)


In [ ]:
t = Perturb.perturb(sentences, add_NiWords)
test = DIR(**t, expect=monotonic_increasing)

In [ ]:
test.run(pred_and_conf)
test.summary()

Add phrases

In [ ]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('You are my {pos_adj}.').data
positive += ['I would like to {neutral_verb_present} if your {parent_nount} not home.']
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert


In [ ]:
t = Perturb.perturb(pdata, add_phrase_function(positive), nsamples=500)
test = MFT(t.data, labels=1)
description = 'Add very predator laden words to the end of sentences, expect probability of negative to go up (tolerance=0.1)'
suite.add(test, 'add positive phrases', 'Vocabulary', description, overwrite=True)


In [ ]:
suite.run(pred_and_conf, overwrite=True)

Running add positive phrases
Predicting 5500 examples


In [ ]:
suite.summary()

**Capability: robustness**

adding irrelevant stuff before and after.


In [ ]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)


def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@user '] + urls_and_handles
    irrelevant_after = urls_and_handles
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets


In [ ]:
t = Perturb.perturb(pdata, add_irrelevant, nsamples=500)
test = INV(t.data)
suite.add(test, 'add random urls and handles', 'Robustness', 'add randomly generated urls and handles to the start or end of sentence', overwrite=True)


punctuation, contractions, typos




In [ ]:
t = Perturb.perturb(pdata, Perturb.punctuation, nsamples=500)
test = INV(t.data)
suite.add(test, 'punctuation', 'Robustness', 'strip punctuation and / or add "."', overwrite=True)


In [ ]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=1)
test = INV(t.data)
suite.add(test, 'typos', 'Robustness', 'Add one typo to input by swapping two adjacent characters', overwrite=True )


In [ ]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=2)
test = INV(t.data)
suite.add(test, '2 typos', 'Robustness', 'Add two typos to input by swapping two adjacent characters twice', overwrite=True )


In [ ]:
t = Perturb.perturb(sentences, Perturb.contractions, nsamples=1000)
test = INV(t.data)
suite.add(test, 'contractions', 'Robustness', 'Contract or expand contractions, e.g. What is -> What\'s', overwrite=True)

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
suite.run(pred_and_conf, overwrite=True)

In [ ]:
suite.summary()

In [ ]:
#suite.visual_summary_table()

**Capability: negation**

In [ ]:
# First, let's find some positive and negative adjectives
', '.join(editor.suggest('This do not want to {a:mask} you.')[:30])

'hurt, to, with, upset, stop, on, provoke, of, for, offend, help, you, attack, abuse, end, insult, embarrass, up, exploit, about, affect, out, change, inconvenience, injure, bother, off, educate, harm, impact'

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
#verb = ['meet', 'call', 'chat', 'talk', 'see', 'exploit']
#ret = editor.template('I do not want to {verb} you.', verb=verb, labels=0, save=True, nsamples=300)

In [ ]:
t = Perturb.perturb(pdata, Perturb.add_negation)
#ret.data

In [ ]:
test = INV(t.data, labels=1)
suite.add(test, 'simple negations', 'Negation', 'Negating predator statements should result in non-predator predictions', overwrite=True )


In [ ]:
suite.run(pred_and_conf)

In [ ]:
suite.summary()

**Capability: taxonomy**

In [ ]:
editor.synonyms('I like young girls.', 'young')
editor.synonyms('I prefer younger girls.', 'younger')

synonyms = [('young', 'underage'), ('young', 'amature'), ('young', 'childish'), ('young', 'little'), ('young', 'teenage')]


In [ ]:
t = editor.template([(
    'Hi, asl? 13 f from New York. You? kewl, Im Mike 40, Florida. I prefer talking to {x[1]} girls, I prefer {x[0]} girls, just like you sweety',
    'I like very {x[0]} girls',
    'I prefer {x[0]} girls',
    ),
    (
    'Hi, asl? 13 f from New York. You? Im Mike 40, Florida. I like {x[1]} girls, wanna chat?',
    'I like very {x[1]} girls',
    'I prefer {x[1]} girls',
    #'DO you prefer {x[1]} girls?',
    )],
    unroll=True,
    x=synonyms,
    remove_duplicates=True,
    padding=True,
    truncation=True,
    nsamples=500,
    overwrite=True)
name = 'replace age describing adj'
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
suite.add(test, overwrite= True)

In [ ]:
suite.run(pred_and_conf, overwrite=True)

In [ ]:
suite.summary()